In [1]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import sklearn.ensemble
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [83]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit import rdmolops

ImportError: cannot import name rdmolops

In [60]:
from rdkit.Chem import rdMolDescriptors as desc
from rdkit.DataStructs import cDataStructs as ds
from rdkit.DataStructs import UIntSparseIntVect as usv

In [4]:
# trainfile = "train.csv"
# testfile = "test.csv"

# data = pd.read_csv(trainfile, nrows=100000)
# names = data['smiles']
# data = data.drop('smiles',1)
# train = data[:50000]
# valid = data[50001:]

In [104]:
trainset = pd.read_csv('orig_train.csv',nrows=10000)
testset = pd.read_csv('orig_test.csv',nrows=10000)
smiles = trainset['smiles']
mid = trainset.shape[0]/2
smiles_t = smiles[:mid]
smiles_v = smiles[mid:]
trainset = trainset.drop('smiles',1)

train = trainset[:mid]
valid = trainset[mid:]

In [82]:
print train.shape
print valid.shape

(500000, 257)
(500000, 257)


In [105]:
ytrain = train['gap']
xtrain = train.drop('gap',1)
yvalid = valid['gap']
xvalid = valid.drop('gap',1)

In [ ]:
# trainfile = "train.csv"
# testfile = "test.csv"

# data = pd.read_csv(trainfile, nrows=100000)
# names = data['smiles']
# data = data.drop('smiles',1)
# train = data[:50000]
# valid = data[50000:]

In [106]:
mols_t = []
for smile in smiles_t:
    m = Chem.MolFromSmiles(smile)
    mols_t.append(m)

In [107]:
mols_v = []
for smile in smiles_v:
    m = Chem.MolFromSmiles(smile)
    mols_v.append(m)

In [35]:
suppl = Chem.SDMolSupplier(smiles_t)
for mol in suppl:
    print mol.GetNumAtoms()

ArgumentError: Python argument types in
    SDMolSupplier.__init__(SDMolSupplier, Series)
did not match C++ signature:
    __init__(_object*, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > fileName, bool sanitize=True, bool removeHs=True, bool strictParsing=True)
    __init__(_object*)

In [88]:
test_fin = []
for mol in mols_t[:100]:
    test_fings = AllChem.RDKFingerprint(mol,fpSize=500)
    test_fin.append(test_fings)


In [89]:
test_fin

In [90]:
binary = []
for mol in test_fin:
    arr = np.zeros((1,2048))
    m = DataStructs.ConvertToNumpyArray(mol,arr)
    binary.append(m)
    print m
    #print m
a = pd.DataFrame(binary)
print a.size
a

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
100


,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


In [29]:
#mols_t_shuff = random.shuffle(mols_t)
#mols_t_cut = mols_t_shuff[:600]
# choice = random.sample(mols_t,5000)
# choice

 ...]

In [29]:
from rdkit.Chem.AtomPairs import Pairs

In [108]:
# Feature engineering using RDkit
fpstrain = [AllChem.RDKFingerprint(mol,fpSize=1000) for mol in mols_t]
fpstest = [AllChem.RDKFingerprint(mol,fpSize=1000) for mol in mols_v]
xfps_train = []
xfps_test = []

In [32]:
sample = AllChem.GetMorganFingerprintAsBitVect(mols_t[1],2)
s2 = Pairs.GetAtomPairFingerprint(mols_t[1])
print s2
#arr = np.zeros((1,))
DataStructs.ConvertToNumpyArray(s2)#,arr)
a = pd.Series(arr)
print a.nonzero()
print a.size

ArgumentError: Python argument types in
    rdkit.DataStructs.cDataStructs.ConvertToNumpyArray(IntSparseIntVect)
did not match C++ signature:
    ConvertToNumpyArray(RDKit::DiscreteValueVect bv, boost::python::api::object destArray)
    ConvertToNumpyArray(ExplicitBitVect bv, boost::python::api::object destArray)

In [109]:
for fp in fpstrain:
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    xfps_train.append(arr)
eng_train = pd.DataFrame(xfps_train)
for fp in fpstest:
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    xfps_test.append(arr)
eng_test = pd.DataFrame(xfps_test)

In [52]:
# eng_train = pd.DataFrame(xfps_train)
# eng_test = pd.DataFrame(xfps_test)

In [110]:
rand_for = sklearn.ensemble.RandomForestRegressor()
rand_for.fit(eng_train,ytrain)
preds_rf = rand_for.predict(eng_test) 

In [111]:
error = mean_squared_error(yvalid,preds_rf)
error

0.049642828394000006

In [78]:
len(fpstest)

0

In [36]:
pd.Series(np.zeros((1,)))

0    0
dtype: float64

In [13]:
xgtrain = xgb.DMatrix(xtrain, label=ytrain)
xgtest = xgb.DMatrix(xvalid,yvalid)

param = {'max_depth':100, 'eta':1, 'silent':0, 'objective':'reg:linear','learning_rate':0.6}
num_round = 10
xg = xgb.train(param, xgtrain, num_round)
#cvresult = xgb.cv(param, xgtrain, num_boost_round=1000, nfold=5, metrics='auc', early_stopping_rounds=50)

preds_xg = xg.predict(xgtest)
error = mean_squared_error(yvalid,preds_xg)
error


0.076641127494400108

In [16]:
avg_pred = (preds_rf+preds_xg)/2

In [17]:
mean_squared_error(yvalid,avg_pred)

0.076373223778163829

preds